In [ ]:
import os
import cv2 
import time
import json
import numpy as np


from lightglue_utils import *
from preprocess_newdataset import *

import CSRansac

## Dataset 전처리

In [ ]:
origin_coordinates, float_origin_coordinates = get_origin_coordinates()
images = get_images()
lengths = get_lens()

## 에러 평가 코드(인접 프레임)

In [ ]:
def matching_keypoints(img0, img1, stabilizing=False):
    # 이미지가 컬러인지 확인하고 그레이스케일로 변환
    if len(img0.shape) == 3:
        img0_gray = cv2.cvtColor(img0, cv2.COLOR_BGR2GRAY)
    else:
        img0_gray = img0

    if len(img1.shape) == 3:
        img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    else:
        img1_gray = img1

    # SURF 객체 생성
    surf = cv2.xfeatures2d.SURF_create()

    # 키포인트와 디스크립터 검출
    keypoints0, descriptors0 = surf.detectAndCompute(img0_gray, None)
    keypoints1, descriptors1 = surf.detectAndCompute(img1_gray, None)

    # BFMatcher 객체 생성 및 KNN 매칭 수행
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(descriptors0, descriptors1, k=2)

    # 좋은 매칭 선별 (Ratio Test)
    good_matches = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append(m)

    # 매칭된 키포인트 좌표 추출
    points0 = np.float32([keypoints0[m.queryIdx].pt for m in good_matches])
    points1 = np.float32([keypoints1[m.trainIdx].pt for m in good_matches])

    return {"points0": points0, "points1": points1}

In [ ]:
epochs = 1

for video_type in range(0, 5):
    # 각종 평가 수치들
    misannotate_errors = []
    pixel_errors = []
    failed_inliers = 0
    zero_inliers = 0
    inlier_rates = 0

    for epoch in range(epochs):
        len_videos = len(origin_coordinates[video_type])  # 동영상의 개수
        coord_list = [[] for _ in range(len_videos)]

        for video_index in range(len_videos):
            len_coord = len(origin_coordinates[video_type][video_index])
            len_images = len(images[video_type][video_index])

            # 이미지의 길이만큼
            _images = images[video_type][video_index]

            coord_list[video_index] = [[] for _ in range(len_images)]

            img0 = _images[0]

            for i in range(len_images):
                x = origin_coordinates[video_type][video_index][i][0]
                y = origin_coordinates[video_type][video_index][i][1]

                if i != len_images - 1:
                    img1 = _images[i + 1]

                    # SURF와 KNN을 사용한 매칭
                    results = matching_keypoints(img0, img1, stabilizing=False)
                    target_keypoint = results["points0"]
                    frame_keypoint = results["points1"]
                    if len(target_keypoint) < 6:
                        failed_inliers += 1
                        continue

                    homography, mask = CSRansac.csransac(target_keypoint, frame_keypoint)
                    if mask <= 0.3:
                        if mask == 0:
                            zero_inliers += 1
                        failed_inliers += 1

                    inlier_rates += mask
                    projected_pts = CSRansac.perspective_transform(np.array([x, y]), homography)

                    coord_list[video_index][i].append(projected_pts)

                    img0 = img1

        # 에러 측정
        misannotate_error, pixel_error = get_errors(coord_list, float_origin_coordinates[video_type], len_coord)

        misannotate_errors.append(misannotate_error)
        pixel_errors.append(pixel_error)

    error1 = sum(misannotate_errors) / len(misannotate_errors)
    error2 = sum(pixel_errors) / len(pixel_errors)
    error3 = inlier_rates / (len(lengths[video_type]) * epochs)

    print("failed_inliers:", failed_inliers)
    print("zero_inliers:", zero_inliers)
    print("misannotate_error:", error1)
    print("pixel_error:", error2)
    print("inlier_rate:", error3)
    print()


(262.99615537328606, 186.28899835987974)


AttributeError: 'tuple' object has no attribute 'type'